In [67]:
import pandas as pd
from textblob import TextBlob
from datetime import timedelta 
from datetime import datetime
import re

In [68]:
df = pd.read_csv('tweets.csv', encoding="utf-8")

In [69]:
df.isnull().sum()

username             0
to                2690
text                93
retweets             0
favorites            0
replies              0
id                   0
permalink            0
author_id            0
date                 0
formatted_date       0
hashtags          9365
mentions          8599
geo               9407
urls              8156
dtype: int64

In [70]:
# drop insignificant columns
df.drop(['username', 'id', 'permalink', 'author_id', 'geo', 'hashtags', 'formatted_date', 'mentions'], axis=1, inplace=True)

In [71]:
df['text'].astype(str)

0       Just test-fired the Superdraco rocket engine. ...
1       Um, while awesome in its own way, that was not...
2       The SpaceX theme song by Total Ghost is just t...
3       The lady doth protest too little. http://finan...
4       The Model X unveiling will be webcast live on ...
                              ...                        
9402    Yeah, very important to provide C/BiPAP device...
9403    Invasive ventilators are for worst case patien...
9404    Exactly. Moreover, all hospitals were given ex...
9405                         Will call when we reach Mars
9406                                              Exactly
Name: text, Length: 9407, dtype: object

In [72]:
df.describe()

,retweets,favorites,replies
count,9407.000000,9.407000e+03,9407.000000
mean,2036.466674,1.588151e+04,438.382162
std,9729.229462,5.129967e+04,1505.300534
min,0.000000,1.000000e+00,0.000000
25%,41.000000,7.190000e+02,37.000000
50%,129.000000,1.821000e+03,89.000000
75%,880.500000,9.192500e+03,311.000000
max,391538.000000,1.711258e+06,51362.000000


In [73]:
df.head()

,to,text,retweets,favorites,replies,date,urls
0,NaN,Just test-fired the Superdraco rocket engine. ...,63,67,13,2012-02-01 16:03:05+00:00,NaN
1,NaN,"Um, while awesome in its own way, that was not...",5,19,12,2012-02-01 09:12:24+00:00,NaN
2,NaN,The SpaceX theme song by Total Ghost is just t...,32,79,14,2012-01-31 16:49:58+00:00,http://www.youtube.com/watch?v=MezkEiS-6jA
3,NaN,The lady doth protest too little. http://finan...,4,2,1,2012-01-28 19:54:40+00:00,http://finance.yahoo.com/news/topless-proteste...
4,NaN,The Model X unveiling will be webcast live on ...,109,23,18,2012-01-28 02:21:03+00:00,http://teslamotors.com


In [74]:
# # parse date as index and convert timezone to EST
df['date'] = pd.to_datetime( df['date'], format='%Y-%m-%d %H:%M:%S')
df['date'] = df['date'].dt.tz_convert('EST')
df.index = df['date']
df.index.rename('D', inplace=True)
df.sort_index()

,to,text,retweets,favorites,replies,date,urls
D,,,,,,,
2012-01-03 01:58:13-05:00,NaN,What everyone really thinks ... RT “@jonlovett...,93,21,3,2012-01-03 01:58:13-05:00,NaN
2012-01-03 16:04:51-05:00,om,@om Respectfully disagree. Larry made awesome ...,10,11,6,2012-01-03 16:04:51-05:00,NaN
2012-01-03 16:28:25-05:00,NaN,Just finished Massie's book on Catherine. An a...,2,11,2,2012-01-03 16:28:25-05:00,NaN
2012-01-06 12:28:01-05:00,NaN,Feb is huge month. Model X world premier and p...,26,16,11,2012-01-06 12:28:01-05:00,NaN
2012-01-06 12:30:01-05:00,NaN,Next month is also when our Dragon spaceship d...,40,12,9,2012-01-06 12:30:01-05:00,NaN
...,...,...,...,...,...,...,...
2020-05-01 17:32:29-05:00,JohnCleese,Reality is its own parrot parody,176,3742,223,2020-05-01 17:32:29-05:00,NaN
2020-05-01 17:34:24-05:00,Bitcoin,,192,5228,185,2020-05-01 17:34:24-05:00,NaN
2020-05-01 17:39:28-05:00,TheOnion,,157,3995,197,2020-05-01 17:39:28-05:00,NaN


In [75]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 9407 entries, 2012-02-01 11:03:05-05:00 to 2020-04-01 23:12:18-05:00
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype              
---  ------     --------------  -----              
 0   to         6717 non-null   object             
 1   text       9314 non-null   object             
 2   retweets   9407 non-null   int64              
 3   favorites  9407 non-null   int64              
 4   replies    9407 non-null   int64              
 5   date       9407 non-null   datetime64[ns, EST]
 6   urls       1251 non-null   object             
dtypes: datetime64[ns, EST](1), int64(3), object(3)
memory usage: 587.9+ KB


In [76]:
# sentiment_analysis
def sentiment_calc(text):
    try:
        return TextBlob(text).sentiment
    except:
        return (0, 0)

df['sentiment'] = df['text'].apply(sentiment_calc)
df['polarity'] = df['sentiment'].apply(lambda x: tuple(x)[0])
df['subjectivity'] = df['sentiment'].apply(lambda x: tuple(x)[1])
df.drop(['sentiment'], axis=1, inplace=True)

In [77]:
# extract info of if url is news
reg = r'https?:\/\/([\w\d.]+)\/[\w\d\/\-\?\=]*'
news_keywords = r'wsj|news|forbes|bloomberg|finance|money|investopedia|marketwatch|cnbc|times|fortune|nasdaq|cnn|huffpost|cnn|usatoday|npr'
def get_domain(url):
    url = str(url)
    result =  re.search(reg, url)
    if result:
        return result.group(1)
    return None

df['url_domain'] = df['urls'].apply(get_domain)
df['url_news'] = df['url_domain'].str.contains(news_keywords)
df['url_news'].fillna(False, inplace=True)

df.drop(['url_domain'], axis=1, inplace=True)

In [78]:
# extract info of if url is video
df['url_video'] = df['urls'].str.contains('video|youtube.com|watch')
df['url_video'].fillna(False, inplace=True)
df.drop(['urls'], axis=1, inplace=True)

In [79]:
# find if tweet is a retweet
df['reply_count'] = df['to'].notnull()
df.drop(['to'], axis=1, inplace=True)

In [80]:
# extract keyword count
tesla = '[Tt]esla'
closely_related_keywords = r'Model [\w\d] | [Tt]ruck|[Ee]lectric|[Ss]olar|[Rr]oof|Semi|[Aa]uto(nomous|pilot|steer(ing)?)?|[Pp]anel|[Ee]nergy|[Mm]egapack | acceleration|top speed|brak(e|ing)'
money_related_keywords = r'[Aa]ffordable|[Cc]heap|[Ee]xpensive|[Pp]rice]|[Ss]tock'
other_related_keywords = r'[Ss]pace|[Mm]ars|[Ll]aunch|AI|neural|Boring Company|Hyperloop|[Ii]terview|boringcompany |drive(ing)|[Tt]axi | |Cars? | cars? |'
def word_match_count(text, pattern):
    if text is None:
        return 0
    text = str(text)
    return len(re.findall(pattern, text))

df['closely_related'] = df['text'].apply(word_match_count, args=(closely_related_keywords,))
df['money_related'] = df['text'].apply(word_match_count, args=(money_related_keywords,))
df['other_related'] = df['text'].apply(word_match_count, args=(other_related_keywords,))
df['tesla'] = df['text'].apply(word_match_count, args=(tesla,))
df.drop(['text'], axis=1, inplace=True)

In [81]:
# convert bool to int
df[['reply_count', 'url_news', 'url_video']] = df[['reply_count', 'url_news', 'url_video']].astype(int)

In [82]:
df.describe()

,retweets,favorites,replies,polarity,subjectivity,url_news,url_video,reply_count,closely_related,money_related,other_related,tesla
count,9407.000000,9.407000e+03,9407.000000,9407.000000,9407.000000,9407.00000,9407.000000,9407.000000,9407.000000,9407.000000,9407.000000,9407.000000
mean,2036.466674,1.588151e+04,438.382162,0.123346,0.338795,0.01063,0.017965,0.714043,0.119485,0.006272,73.913894,0.112576
std,9729.229462,5.129967e+04,1505.300534,0.266251,0.311150,0.10256,0.132832,0.451893,0.431792,0.084165,62.728419,0.360408
min,0.000000,1.000000e+00,0.000000,-1.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000
25%,41.000000,7.190000e+02,37.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,21.000000,0.000000
50%,129.000000,1.821000e+03,89.000000,0.000000,0.333333,0.00000,0.000000,1.000000,0.000000,0.000000,60.000000,0.000000
75%,880.500000,9.192500e+03,311.000000,0.250000,0.567354,0.00000,0.000000,1.000000,0.000000,0.000000,115.000000,0.000000
max,391538.000000,1.711258e+06,51362.000000,1.000000,1.000000,1.00000,1.000000,1.000000,7.000000,2.000000,445.000000,3.000000


In [83]:
df['day_of_week'] = df['date'].dt.dayofweek

In [84]:
df['time_of_day'] = df['date'].apply(lambda x: int(x.hour))

In [85]:
df_shift = df.copy()
df_shift['date'] = df_shift['date'].apply(lambda x: datetime(x.year, x.month, x.day, tzinfo=x.tzinfo))
for delta in range(1, 4):
    df_shift['date'] = df_shift['date'].apply(lambda x: x + timedelta(days=1))
    df_shift_x = df_shift.copy()
    sums_by_date = df_shift_x[['date', 'retweets', 'favorites', 'replies', 'reply_count', 'closely_related',  'url_news', 'url_video', 'money_related', 'other_related', 'tesla']].groupby(['date']).sum()
    means_by_date = df_shift_x[['date', 'polarity', 'subjectivity', 'time_of_day']].groupby(['date']).mean()
    df_shift_x = pd.merge(sums_by_date, means_by_date, on='date')
    df_shift_x['day_of_week'] = df_shift_x.index.dayofweek
    df_shift_x.to_pickle('tweets_processed_shift_{}.pkl'.format(delta))
    

In [86]:
def shift_if_after_market(date):
    closing_time = datetime(date.year, date.month, date.day, 16, 0, tzinfo=date.tzinfo)
    if date >= closing_time:
        return date + timedelta(days=1)
    return date

df['date'] = df['date'].apply(shift_if_after_market)
df['date'] = df['date'].apply(lambda x: datetime(x.year, x.month, x.day, tzinfo=x.tzinfo))

In [87]:
sums_by_date = df[['date', 'retweets', 'favorites', 'replies', 'reply_count', 'closely_related',  'url_news', 'url_video', 'money_related', 'other_related', 'tesla']].groupby(['date']).sum()
means_by_date = df[['date', 'polarity', 'subjectivity', 'time_of_day']].groupby(['date']).mean()
df2 = pd.merge(sums_by_date, means_by_date, on='date')
df2.describe()

,retweets,favorites,replies,reply_count,closely_related,url_news,url_video,money_related,other_related,tesla,polarity,subjectivity,time_of_day
count,1695.000000,1.695000e+03,1695.000000,1695.000000,1695.000000,1695.000000,1695.000000,1695.000000,1695.000000,1695.000000,1695.000000,1695.000000,1695.000000
mean,11302.089676,8.814006e+04,2432.956342,3.962832,0.663127,0.058997,0.099705,0.034808,410.211209,0.624779,0.134157,0.346711,13.209956
std,27626.406566,1.795665e+05,5202.031098,5.581883,1.566067,0.243087,0.343733,0.213126,543.434391,1.288712,0.193205,0.218366,5.304266
min,0.000000,1.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,-0.800000,0.000000,0.000000
25%,444.000000,1.179500e+03,137.500000,1.000000,0.000000,0.000000,0.000000,0.000000,104.000000,0.000000,0.000000,0.204514,10.055556
50%,2491.000000,1.429700e+04,666.000000,2.000000,0.000000,0.000000,0.000000,0.000000,229.000000,0.000000,0.106647,0.336264,14.000000
75%,9037.000000,8.625750e+04,2651.000000,5.000000,1.000000,0.000000,0.000000,0.000000,498.000000,1.000000,0.214352,0.475000,17.000000
max,415834.000000,1.730902e+06,97657.000000,52.000000,21.000000,2.000000,4.000000,3.000000,6049.000000,14.000000,1.000000,1.000000,23.000000


In [88]:
df2['day_of_week'] = df2.index.dayofweek
df2['date'] = df2.index
df2

,retweets,favorites,replies,reply_count,closely_related,url_news,url_video,money_related,other_related,tesla,polarity,subjectivity,time_of_day,day_of_week,date
date,,,,,,,,,,,,,,,
2012-01-03 00:00:00-05:00,93,21,3,0,0,0,0,0,114,0,0.300000,0.308333,1.000000,1,2012-01-03 00:00:00-05:00
2012-01-04 00:00:00-05:00,12,22,8,1,0,0,0,0,262,0,0.495000,0.661944,16.000000,2,2012-01-04 00:00:00-05:00
2012-01-06 00:00:00-05:00,116,37,38,0,2,0,0,0,279,0,0.134722,0.255556,13.000000,4,2012-01-06 00:00:00-05:00
2012-01-11 00:00:00-05:00,68,325,28,0,0,0,0,0,117,1,0.350000,0.650000,21.000000,2,2012-01-11 00:00:00-05:00
2012-01-12 00:00:00-05:00,110,30,14,0,1,0,0,0,125,0,0.050000,0.400000,21.000000,3,2012-01-12 00:00:00-05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-04-28 00:00:00-05:00,15660,157319,4715,4,0,0,0,0,219,0,0.198611,0.305556,17.833333,1,2020-04-28 00:00:00-05:00
2020-04-29 00:00:00-05:00,98006,581451,63213,3,0,1,0,0,170,0,0.162500,0.283333,4.333333,2,2020-04-29 00:00:00-05:00
2020-04-30 00:00:00-05:00,99374,510522,34649,14,0,0,0,0,507,0,0.073173,0.171984,14.842105,3,2020-04-30 00:00:00-05:00


In [89]:
def date_alt(df):
    date = df['date']
    dayofweek = df['day_of_week']
    if (dayofweek == 6):
        return date + timedelta(days=1)
    if (dayofweek == 5):
        return date + timedelta(days=2)
    return date

df2['date'] = df2[['date', 'day_of_week']].apply(date_alt, axis=1) 
df2.index = df2['date']

In [90]:
sums_by_date = df[['date', 'retweets', 'favorites', 'replies', 'reply_count', 'closely_related',  'url_news', 'url_video', 'money_related', 'other_related', 'tesla']].groupby(['date']).mean()
means_by_date = df[['date', 'polarity', 'subjectivity', 'time_of_day']].groupby(['date']).mean()
df2 = pd.merge(sums_by_date, means_by_date, on='date')

In [91]:
df2['day_of_week'] = df2.index.dayofweek
df2

,retweets,favorites,replies,reply_count,closely_related,url_news,url_video,money_related,other_related,tesla,polarity,subjectivity,time_of_day,day_of_week
date,,,,,,,,,,,,,,
2012-01-03 00:00:00-05:00,93.000000,21.000000,3.000000,0.000000,0.000000,0.000000,0.0,0.0,114.000000,0.000000,0.300000,0.308333,1.000000,1
2012-01-04 00:00:00-05:00,6.000000,11.000000,4.000000,0.500000,0.000000,0.000000,0.0,0.0,131.000000,0.000000,0.495000,0.661944,16.000000,2
2012-01-06 00:00:00-05:00,38.666667,12.333333,12.666667,0.000000,0.666667,0.000000,0.0,0.0,93.000000,0.000000,0.134722,0.255556,13.000000,4
2012-01-11 00:00:00-05:00,68.000000,325.000000,28.000000,0.000000,0.000000,0.000000,0.0,0.0,117.000000,1.000000,0.350000,0.650000,21.000000,2
2012-01-12 00:00:00-05:00,110.000000,30.000000,14.000000,0.000000,1.000000,0.000000,0.0,0.0,125.000000,0.000000,0.050000,0.400000,21.000000,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-04-28 00:00:00-05:00,2610.000000,26219.833333,785.833333,0.666667,0.000000,0.000000,0.0,0.0,36.500000,0.000000,0.198611,0.305556,17.833333,1
2020-04-29 00:00:00-05:00,16334.333333,96908.500000,10535.500000,0.500000,0.000000,0.166667,0.0,0.0,28.333333,0.000000,0.162500,0.283333,4.333333,2
2020-04-30 00:00:00-05:00,5230.210526,26869.578947,1823.631579,0.736842,0.000000,0.000000,0.0,0.0,26.684211,0.000000,0.073173,0.171984,14.842105,3


In [92]:
df2.describe()

,retweets,favorites,replies,reply_count,closely_related,url_news,url_video,money_related,other_related,tesla,polarity,subjectivity,time_of_day,day_of_week
count,1695.000000,1695.000000,1695.000000,1695.000000,1695.000000,1695.000000,1695.000000,1695.000000,1695.000000,1695.000000,1695.000000,1695.000000,1695.000000,1695.000000
mean,2094.810491,14329.607366,408.619648,0.578125,0.144823,0.024119,0.029468,0.005611,77.059828,0.143522,0.134157,0.346711,13.209956,2.966962
std,6315.779697,34455.923660,876.388459,0.389469,0.345860,0.128907,0.132713,0.058481,38.179404,0.308986,0.193205,0.218366,5.304266,1.974177
min,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,-0.800000,0.000000,0.000000,0.000000
25%,199.250000,676.500000,71.900000,0.126984,0.000000,0.000000,0.000000,0.000000,48.212121,0.000000,0.000000,0.204514,10.055556,1.000000
50%,716.800000,4012.000000,190.647059,0.666667,0.000000,0.000000,0.000000,0.000000,75.500000,0.000000,0.106647,0.336264,14.000000,3.000000
75%,1778.750000,14750.087662,442.666667,1.000000,0.125000,0.000000,0.000000,0.000000,103.000000,0.142857,0.214352,0.475000,17.000000,5.000000
max,170455.500000,865451.000000,23664.500000,1.000000,5.000000,1.000000,1.000000,2.000000,262.000000,3.000000,1.000000,1.000000,23.000000,6.000000


In [93]:
# cat_col = df2[['retweets', 'favorites', 'replies', 'url_news', 'url_video', 'is_retweet', 'closely_related', 'money_related', 'other_related', 'tesla', 'day_of_week', 'time_of_day']]
# cat_col = catcol.astype('category')
# val_col = df2[['polarity', 'subjectivity']]
# df3 = pd.merge(cat_col, val_col, on='date')
df2.dtypes

retweets           float64
favorites          float64
replies            float64
reply_count        float64
closely_related    float64
url_news           float64
url_video          float64
money_related      float64
other_related      float64
tesla              float64
polarity           float64
subjectivity       float64
time_of_day        float64
day_of_week          int64
dtype: object

In [62]:
df2.to_pickle('tweets_processed.pkl')

In [64]:
df2.describe()

,retweets,favorites,replies,reply_count,closely_related,url_news,url_video,money_related,other_related,tesla,polarity,subjectivity,time_of_day,day_of_week
count,1695.000000,1695.000000,1695.000000,1695.000000,1695.000000,1695.000000,1695.000000,1695.000000,1695.000000,1695.000000,1695.000000,1695.000000,1695.000000,1695.000000
mean,2094.810491,14329.607366,408.619648,0.578125,0.144823,0.024119,0.029468,0.005611,77.059828,0.143522,0.134157,0.346711,13.209956,2.966962
std,6315.779697,34455.923660,876.388459,0.389469,0.345860,0.128907,0.132713,0.058481,38.179404,0.308986,0.193205,0.218366,5.304266,1.974177
min,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,-0.800000,0.000000,0.000000,0.000000
25%,199.250000,676.500000,71.900000,0.126984,0.000000,0.000000,0.000000,0.000000,48.212121,0.000000,0.000000,0.204514,10.055556,1.000000
50%,716.800000,4012.000000,190.647059,0.666667,0.000000,0.000000,0.000000,0.000000,75.500000,0.000000,0.106647,0.336264,14.000000,3.000000
75%,1778.750000,14750.087662,442.666667,1.000000,0.125000,0.000000,0.000000,0.000000,103.000000,0.142857,0.214352,0.475000,17.000000,5.000000
max,170455.500000,865451.000000,23664.500000,1.000000,5.000000,1.000000,1.000000,2.000000,262.000000,3.000000,1.000000,1.000000,23.000000,6.000000
